In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.0 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from datasets import Dataset
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# ds = load_dataset("mahfoos/Patient-Doctor-Conversation")

In [ ]:
# ds

In [ ]:
# df = pd.read_csv("/content/drive/MyDrive/PROJECTS/AI Med Bot/severity.csv")
df = pd.read_csv("/content/drive/MyDrive/Machine Learning/AI Med Bot/severity.csv")

df['Status'] = df['Status'] - 1
df.rename(columns={'Status': 'labels'}, inplace=True)
df

,Patient,labels
0,hi doctor I am just wondering what is abutting...,1
1,hi doctor I am a 26 year old male I am feet an...,2
2,hello doctor I am 48 years old I am experienci...,0
3,hello doctor I have multiple small cysts in bo...,1
4,hi doctor during masturbation I just rub the t...,0
...,...,...
3320,hello doctor I am currently seeking help and I...,2
3321,hi doctor I fell on a wooden set of stairs las...,1
3322,hi doctor my mother is 57 years old she has re...,0
3323,hi doctor my 18 month old daughter has had pro...,1


In [ ]:
seq_len = 512
num_samples = len(df)


Xids =  np.zeros((num_samples, seq_len))
Xmask = np.zeros((num_samples, seq_len))

Xids.shape

(3325, 512)

In [ ]:
arr = df['labels'].values
arr

array([1, 2, 0, ..., 0, 1, 0])

In [ ]:
labels = np.zeros((num_samples, 3 ))
labels.shape

(3325, 3)

In [ ]:
labels[np.arange(num_samples), arr] = 1
labels

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_encoder = TFAutoModel.from_pretrained("bert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
for i, text in enumerate(df['Patient']):
  tokens = bert_tokenizer.encode_plus(text, max_length = seq_len,
                                      truncation = True, padding = 'max_length',
                                      add_special_tokens=True, return_tensors = 'tf')
  Xids[i, :] = tokens['input_ids']
  Xmask[i, :] = tokens['attention_mask']

In [ ]:
l = [Xids, Xmask]
l[0]
# 101.,  7632.,  3460.,  1045.,  2031.,  1037., 14182., 23438.,
      #   2006.,  2026.,  5110., 10120.,  2144.,  2706.,  1045.,  2031.,
      #   4987.,  1037.,  3861.,  2005.,  2115.,  4431.,  2009.,  2003.,
      #  15703.,  2033.,  1037.,  2843.,  3531.,  2393.,   102.,

array([[ 101., 7632., 3460., ...,    0.,    0.,    0.],
       [ 101., 7632., 3460., ...,    0.,    0.,    0.],
       [ 101., 7592., 3460., ...,    0.,    0.,    0.],
       ...,
       [ 101., 7632., 3460., ...,    0.,    0.,    0.],
       [ 101., 7632., 3460., ..., 3426., 2057.,  102.],
       [ 101., 7632., 3460., ...,    0.,    0.,    0.]])

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

In [ ]:
def map_func(input_ids, masks, labels):
  return {'input_ids': input_ids,
          'attention_mask': masks}, labels

In [ ]:
dataset = dataset.map(map_func)
dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.float64, name=None)}, TensorSpec(shape=(3,), dtype=tf.float64, name=None))>

In [ ]:
BATCH_SIZE = 8

dataset = dataset.shuffle(100).batch(BATCH_SIZE, drop_remainder=True)
dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(8, 512), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(8, 512), dtype=tf.float64, name=None)}, TensorSpec(shape=(8, 3), dtype=tf.float64, name=None))>

In [ ]:
split = 0.9

size = int((num_samples / BATCH_SIZE) * split)
size

374

In [ ]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)
val_ds

<_SkipDataset element_spec=({'input_ids': TensorSpec(shape=(8, 512), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(8, 512), dtype=tf.float64, name=None)}, TensorSpec(shape=(8, 3), dtype=tf.float64, name=None))>

In [ ]:
768/2

384.0

In [ ]:
input_ids = tf.keras.layers.Input(shape=(seq_len,), name='input_ids', dtype = 'int32')
mask = tf.keras.layers.Input(shape=(seq_len, ), name='attention_mask', dtype='int32')

embeddings = bert_encoder.bert(input_ids, attention_mask = mask)[1]
# x1 = tf.keras.layers.Dense(1024, activation='relu')(embeddings)
n = tf.keras.layers.Normalization(embeddings)
d = tf.keras.layers.Dropout(0.1)(n)
# x = tf.keras.layers.Dense(512, activation='relu')(d)
y = tf.keras.layers.Dense(3, activation = 'softmax', name='outputs')(d)


model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)



TypeError: ignored

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  335141888   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                         

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy()
acc =  tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(
    optimizer= optimizer ,
    loss= loss ,
    metrics= [acc]
)

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=4
)

Epoch 1/4


ResourceExhaustedError: ignored

In [ ]:
model.save("/content/drive/MyDrive/Machine Learning/AI Med Bot/model3")

In [ ]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/PROJECTS/AI Med Bot/model')

In [ ]:
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

In [ ]:
x = bert_encoder(test_dataset)

In [ ]:
new_model = tf.keras.models.load_model('/content/severity.keras')

In [ ]:
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def prep_data(text):
  tokens = tokenizer.encode_plus(text, max_length=512,
                                 truncation=True, padding='max_length',
                                 add_special_tokens=True, return_token_type_ids=False,
                                 return_tensors = 'tf')
  return {
      'input_ids': tf.cast(tokens['input_ids'], tf.float64),
      'attention_mask': tf.cast(tokens['attention_mask'], tf.float64)
  }

In [ ]:
test = prep_data("I'm having fun")

In [ ]:
probs = new_model.predict(test)
probs[0]

1/1 [==============================] - 0s 116ms/step


array([9.5140290e-01, 4.7718268e-02, 8.7890908e-04], dtype=float32)

In [ ]:
np.argmax(probs[0])

0

In [ ]:
df['labels'].value_counts()

1    1780
0    1072
2     473
Name: labels, dtype: int64